## AI Finance Project - Loda Enrico

In [1]:
!pip install --upgrade yfinance

In [2]:
# import
import pandas as pd
import yfinance as yf
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

We want to calculate the daily simple rate of return of our assets. To do so, we define:

---

**Simple rate of return**  
Given:  
- **P**: the price of the asset at the time of investment  
- **T**: maturity (number of periods)  
- **C**: the price of the asset at maturity **T**

The formula is:
$$
r = \frac{\frac{C}{P} - 1}{T}
$$

---

**Daily simple rate of return:**  
For daily returns, considering $P_t$ as the close price of the asset at time t, the formula is:
$$
r_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1
$$

In [3]:
df = yf.download(["SPY","NVDA"], start="2010-01-01", end="2025-05-17")
spy = df.xs('SPY', axis=1, level=1)
nvda = df.xs('NVDA', axis=1, level=1)

prices = df["Close"]
returns = (prices / prices.shift(1)) - 1
returns = returns.dropna()
SPY_r = returns['SPY']
NVDA_r = returns['NVDA']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


In [4]:
df

Price            Close                    High                     Low  \
Ticker            NVDA         SPY        NVDA         SPY        NVDA   
Date                                                                     
2010-01-04    0.423884   85.768417    0.426864   85.813824    0.415172   
2010-01-05    0.430073   85.995468    0.434658   86.033311    0.422279   
2010-01-06    0.432824   86.056046    0.433741   86.267949    0.425718   
2010-01-07    0.424342   86.419304    0.432366   86.525256    0.421133   
2010-01-08    0.425259   86.706863    0.428239   86.744706    0.418382   
...                ...         ...         ...         ...         ...   
2025-05-12  123.000000  582.989990  123.000000  583.000000  120.279999   
2025-05-13  129.929993  586.840027  131.220001  589.080017  124.470001   
2025-05-14  135.339996  587.590027  135.440002  588.979980  131.679993   
2025-05-15  134.830002  590.460022  136.300003  590.969971  132.660004   
2025-05-16  135.399994  594.200012  136.350006  594.500000  133.460007   

Price                         Open                 Volume             
Ticker             SPY        NVDA         SPY       NVDA        SPY  
Date                                                                  
2010-01-04   84.391037    0.424342   85.041888  800204000  118944600  
2010-01-05   85.405163    0.422279   85.715455  728648000  111579900  
2010-01-06   85.844142    0.429844   85.912251  649168000  116074400  
2010-01-07   85.654932    0.430532   85.897108  547792000  131091100  
2010-01-08   86.018176    0.420903   86.192237  478168000  126402800  
...                ...         ...         ...        ...        ...  
2025-05-12  577.039978  121.970001  581.469971  225023300   78993600  
2025-05-13  582.840027  124.980003  583.409973  330430100   67947200  
2025-05-14  585.539978  133.199997  587.809998  281180800   66283500  
2025-05-15  585.099976  134.289993  585.559998  226632600   71268100  
2025-05-16  589.280029  136.220001  591.250000  226542500   76052100  

[3867 rows x 10 columns]

In [5]:
returns[:]

Ticker,NVDA,SPY
Date,,
2010-01-05,0.014603,0.002647
2010-01-06,0.006396,0.000704
2010-01-07,-0.019597,0.004221
2010-01-08,0.002161,0.003327
2010-01-11,-0.014016,0.001397
...,...,...
2025-05-12,0.054436,0.033047
2025-05-13,0.056341,0.006604
2025-05-14,0.041638,0.001278


#dataset for training
The features, for a total of 24, are the following:

*   $\mu_1$, $\mu_2$ of $r_1$, $r_2$ calculated on the last 10 lags
*   $[r_1^t,\dots ,r_1^{t-10}]$, $[r_2^t,\dots ,r_2^{t-10}]$

The target is the values of my covariance matrix:
* $\Sigma_{t+1}$

In [6]:
rNVDA_series = returns['NVDA']
rSPY_series = returns['SPY']
df = pd.DataFrame()
df['rNVDA_today'] = rNVDA_series
df['rSPY_today'] = rSPY_series

for i in range(1, 11):
    df[f'rNVDA_t-{i}'] = rNVDA_series.shift(i)
    df[f'rSPY_t-{i}'] = rSPY_series.shift(i)

df.head(11)

,rNVDA_today,rSPY_today,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,...,rNVDA_t-6,rSPY_t-6,rNVDA_t-7,rSPY_t-7,rNVDA_t-8,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.014603,0.002647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.006396,0.000704,0.014603,0.002647,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,-0.019597,0.004221,0.006396,0.000704,0.014603,0.002647,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.002161,0.003327,-0.019597,0.004221,0.006396,0.000704,0.014603,0.002647,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,-0.014016,0.001397,0.002161,0.003327,-0.019597,0.004221,0.006396,0.000704,0.014603,0.002647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-12,-0.033898,-0.009327,-0.014016,0.001397,0.002161,0.003327,-0.019597,0.004221,0.006396,0.000704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-13,0.013582,0.008447,-0.033898,-0.009327,-0.014016,0.001397,0.002161,0.003327,-0.019597,0.004221,...,0.014603,0.002647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-14,-0.015634,0.002704,0.013582,0.008447,-0.033898,-0.009327,-0.014016,0.001397,0.002161,0.003327,...,0.006396,0.000704,0.014603,0.002647,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,-0.029495,-0.011224,-0.015634,0.002704,0.013582,0.008447,-0.033898,-0.009327,-0.014016,0.001397,...,-0.019597,0.004221,0.006396,0.000704,0.014603,0.002647,NaN,NaN,NaN,NaN


In [7]:
cols = [f'rNVDA_t-{i}' for i in range(1, 11)]
df['mean_NVDA'] = df[cols].mean(axis=1)
cols.append('rNVDA_today')
df['var_NVDA'] = df[cols].var(axis=1)

cols = [f'rSPY_t-{i}' for i in range(1, 11)]
df['mean_SPY'] = df[cols].mean(axis=1)
cols.append('rSPY_today')
df['var_SPY'] = df[cols].var(axis=1)
df = df.dropna()
df#["r2_t-10"]

,rNVDA_today,rSPY_today,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,...,rNVDA_t-8,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,0.013582,0.008447,...,-0.019597,0.004221,0.006396,0.000704,0.014603,0.002647,-0.005720,0.000331,0.001539,0.000059
2010-01-21,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,...,0.002161,0.003327,-0.019597,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093
2010-01-22,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,...,-0.014016,0.001397,0.002161,0.003327,-0.019597,0.004221,-0.010007,0.000327,-0.001736,0.000131
2010-01-25,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,...,-0.033898,-0.009327,-0.014016,0.001397,0.002161,0.003327,-0.011508,0.000395,-0.004387,0.000132
2010-01-26,-0.031660,-0.004190,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,...,0.013582,0.008447,-0.033898,-0.009327,-0.014016,0.001397,-0.010023,0.000424,-0.004207,0.000127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-12,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,-0.002460,-0.008358,...,-0.000917,0.000397,0.002667,0.006299,-0.020539,0.000381,0.005092,0.000471,0.002482,0.000127
2025-05-13,0.056341,0.006604,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,...,0.024697,0.007087,-0.000917,0.000397,0.002667,0.006299,0.012589,0.000545,0.005748,0.000124
2025-05-14,0.041638,0.001278,0.056341,0.006604,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,...,0.025894,0.014844,0.024697,0.007087,-0.000917,0.000397,0.017957,0.000575,0.005779,0.000126


In [8]:
def compute_covariance_row(row):
    r1 = [row[f'rNVDA_t-{i}'] for i in range(1, 11)]
    r1.append(row['rNVDA_today'])

    r2 = [row[f'rSPY_t-{i}'] for i in range(1, 11)]
    r2.append(row['rSPY_today'])

    return np.cov(r1, r2)[0, 1]

df['cov'] = df.apply(compute_covariance_row, axis=1)
df

,rNVDA_today,rSPY_today,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,...,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY,cov
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,0.013582,0.008447,...,0.004221,0.006396,0.000704,0.014603,0.002647,-0.005720,0.000331,0.001539,0.000059,0.000097
2010-01-21,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,...,0.003327,-0.019597,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093,0.000111
2010-01-22,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,...,0.001397,0.002161,0.003327,-0.019597,0.004221,-0.010007,0.000327,-0.001736,0.000131,0.000153
2010-01-25,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,...,-0.009327,-0.014016,0.001397,0.002161,0.003327,-0.011508,0.000395,-0.004387,0.000132,0.000184
2010-01-26,-0.031660,-0.004190,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,...,0.008447,-0.033898,-0.009327,-0.014016,0.001397,-0.010023,0.000424,-0.004207,0.000127,0.000175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-12,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,-0.002460,-0.008358,...,0.000397,0.002667,0.006299,-0.020539,0.000381,0.005092,0.000471,0.002482,0.000127,0.000202
2025-05-13,0.056341,0.006604,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,...,0.007087,-0.000917,0.000397,0.002667,0.006299,0.012589,0.000545,0.005748,0.000124,0.000190
2025-05-14,0.041638,0.001278,0.056341,0.006604,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,...,0.014844,0.024697,0.007087,-0.000917,0.000397,0.017957,0.000575,0.005779,0.000126,0.000181


In [9]:
df_r = df[['rNVDA_today', 'rSPY_today']]
df.drop(['rNVDA_today', 'rSPY_today'], axis=1, inplace=True)
df

,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,rNVDA_t-5,rSPY_t-5,...,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY,cov
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,0.013582,0.008447,-0.033898,-0.009327,...,0.004221,0.006396,0.000704,0.014603,0.002647,-0.005720,0.000331,0.001539,0.000059,0.000097
2010-01-21,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,0.013582,0.008447,...,0.003327,-0.019597,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093,0.000111
2010-01-22,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,-0.015634,0.002704,...,0.001397,0.002161,0.003327,-0.019597,0.004221,-0.010007,0.000327,-0.001736,0.000131,0.000153
2010-01-25,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,-0.029495,-0.011224,...,-0.009327,-0.014016,0.001397,0.002161,0.003327,-0.011508,0.000395,-0.004387,0.000132,0.000184
2010-01-26,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018702,0.012495,...,0.008447,-0.033898,-0.009327,-0.014016,0.001397,-0.010023,0.000424,-0.004207,0.000127,0.000175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-12,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,-0.002460,-0.008358,-0.005939,-0.005734,...,0.000397,0.002667,0.006299,-0.020539,0.000381,0.005092,0.000471,0.002482,0.000127,0.000202
2025-05-13,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,-0.002460,-0.008358,...,0.007087,-0.000917,0.000397,0.002667,0.006299,0.012589,0.000545,0.005748,0.000124,0.000190
2025-05-14,0.056341,0.006604,0.054436,0.033047,-0.006134,-0.001274,0.002648,0.006968,0.031002,0.004206,...,0.014844,0.024697,0.007087,-0.000917,0.000397,0.017957,0.000575,0.005779,0.000126,0.000181


### we are trying to scale in order to have numerical stability, we are not sure it is correct since we need to have the non-scaled version of the variance at inference time

In [10]:
train_df = df.loc["2010-01-01":"2023-12-31"]
test_df  = df.loc["2024-01-01":]
print((len(train_df) + len(test_df)) == len(df))


##CARE FOR SCALER
#scaler = StandardScaler()
#scaler.fit(train_df)
#train_df = pd.DataFrame(scaler.transform(train_df), columns=train_df.columns, index=train_df.index)
#test_df = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns, index=test_df.index)

True


Flip the columns in order to have a temporal sequence from past to present

In [11]:
train_df = train_df[train_df.columns[::-1]]
test_df = test_df[test_df.columns[::-1]]

In [12]:
train_df

,cov,var_SPY,mean_SPY,var_NVDA,mean_NVDA,rSPY_t-10,rNVDA_t-10,rSPY_t-9,rNVDA_t-9,rSPY_t-8,...,rSPY_t-5,rNVDA_t-5,rSPY_t-4,rNVDA_t-4,rSPY_t-3,rNVDA_t-3,rSPY_t-2,rNVDA_t-2,rSPY_t-1,rNVDA_t-1
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,0.000097,0.000059,0.001539,0.000331,-0.005720,0.002647,0.014603,0.000704,0.006396,0.004221,...,-0.009327,-0.033898,0.008447,0.013582,0.002704,-0.015634,-0.011224,-0.029495,0.012495,0.018702
2010-01-21,0.000111,0.000093,0.000258,0.000296,-0.007581,0.000704,0.006396,0.004221,-0.019597,0.003327,...,0.008447,0.013582,0.002704,-0.015634,-0.011224,-0.029495,0.012495,0.018702,-0.010168,-0.004016
2010-01-22,0.000153,0.000131,-0.001736,0.000327,-0.010007,0.004221,-0.019597,0.003327,0.002161,0.001397,...,0.002704,-0.015634,-0.011224,-0.029495,0.012495,0.018702,-0.010168,-0.004016,-0.019229,-0.017857
2010-01-25,0.000184,0.000132,-0.004387,0.000395,-0.011508,0.003327,0.002161,0.001397,-0.014016,-0.009327,...,-0.011224,-0.029495,0.012495,0.018702,-0.010168,-0.004016,-0.019229,-0.017857,-0.022292,-0.034604
2010-01-26,0.000175,0.000127,-0.004207,0.000424,-0.010023,0.001397,-0.014016,-0.009327,-0.033898,0.008447,...,0.012495,0.018702,-0.010168,-0.004016,-0.019229,-0.017857,-0.022292,-0.034604,0.005128,0.017011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,0.000073,0.000048,0.003544,0.000316,0.005178,0.004299,0.019530,0.003890,-0.018503,0.004567,...,-0.001647,0.011169,0.005625,0.024279,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270
2023-12-26,0.000072,0.000048,0.003315,0.000295,0.002899,0.003890,-0.018503,0.004567,0.022090,0.013790,...,0.005625,0.024279,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266
2023-12-27,0.000074,0.000049,0.003348,0.000242,0.005669,0.004567,0.022090,0.013790,0.009044,0.003209,...,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266,0.004223,0.009195


In [13]:
scaler_feature  = StandardScaler()
scaler_target = StandardScaler()

class LSTMdataset(Dataset):
    def __init__(self, dataframe, train):
        super().__init__()

        feature_dataframe = dataframe.drop(['cov', 'var_NVDA', 'var_SPY'], axis=1)
        target_dataframe = dataframe[['cov', 'var_NVDA', 'var_SPY']]

        if train:
          feature_dataframe = scaler_feature.fit_transform(feature_dataframe)
          target_dataframe = scaler_target.fit_transform(target_dataframe)
        else:
          feature_dataframe = scaler_feature.transform(feature_dataframe)
          target_dataframe = scaler_target.transform(target_dataframe)

        self.X = torch.tensor(feature_dataframe, dtype=torch.float32)
        self.y = torch.tensor(target_dataframe, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [14]:
# post normalization
train_dataset = LSTMdataset(train_df, train=True)
train_dataset[3]


# Get all targets from the dataset as a tensor
all_targets = train_dataset[:][1]  # shape: (num_samples, 3)

# Convert to numpy array
all_targets_np = all_targets.numpy()

# Print min and max for each column (cov, var_NVDA, var_SPY)
col_names = ['cov', 'var_NVDA', 'var_SPY']
for i, name in enumerate(col_names):
    print(f"{name}: min={all_targets_np[:, i].min():.6f}, max={all_targets_np[:, i].max():.6f}")

cov: min=-0.887613, max=16.611055
var_NVDA: min=-0.748865, max=10.517368
var_SPY: min=-0.412288, max=16.399763


In [15]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
x, y = next(iter(train_loader))
print(x.shape, y.shape)

torch.Size([32, 22]) torch.Size([32, 3])


In [16]:
# batch first lets us have the tensor in the shape (batch=32, sequence=10, feature=2)
class LSTM(torch.nn.Module):
    def __init__(self, input_size=2, hidden_size=20, num_layers=2, output_size=3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm  = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.3
          )
        # hidden size + 2 since the fc will take in input the mean of each asset
        self.fc_1 = torch.nn.Linear (hidden_size+2, (hidden_size+2)//2)
        self.dropout = torch.nn.Dropout(0.3)
        self.fc_2 = torch.nn.Linear ((hidden_size+2)//2, output_size)

    # x_lstm = (32, 10, 2) [N,L,H_in]
    # x_means = (32,2)
    def forward (self, x_lstm, x_means):

      output_lstm, _ = self.lstm(x_lstm)

      # now x_lstm = (32, 10, hidden_size)[N,L,Hout​]
      # here we take only the last hidden state
      last_hidden_state = output_lstm[:,-1,:]
      # now last_hidden_state should be (32,hidden_size)

      x_cat = torch.cat([last_hidden_state, x_means], dim=1)
      x_out = self.fc_1(x_cat)
      x_out = torch.relu(x_out)
      x_out = self.dropout(x_out)
      x_out = self.fc_2(x_out)
      return x_out

In [17]:
# Hyperparameters
batch_size = 32
learning_rate = 1e-3
#learning_rate = 1e-4
epochs = 50

# Prepare datasets and dataloaders
train_dataset = LSTMdataset(train_df, train=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Device setup: CUDA > MPS (Mac) > CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Model, loss, optimizer
model = LSTM(input_size=2, hidden_size=20, num_layers=3, output_size=3).to(device)
criterion = torch.nn.MSELoss()
#criterion = torch.nn.L1Loss()

# weight_decay implement L2 norm
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        # X: (batch, features), need to reshape for LSTM: (batch, seq_len, input_size)
        # X shape: (batch, 24): first 2 are means, next 20 are lags
        x_lstm = X[:, 2:].reshape(-1, 10, 2)
        x_means = X[:, :2]

        optimizer.zero_grad()
        outputs = model(x_lstm, x_means)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.6f}")

Using device: cuda
Epoch 1/50, Loss: 1.005427
Epoch 2/50, Loss: 0.982213
Epoch 3/50, Loss: 0.908848
Epoch 4/50, Loss: 0.758788
Epoch 5/50, Loss: 0.680591
Epoch 6/50, Loss: 0.586056
Epoch 7/50, Loss: 0.496008
Epoch 8/50, Loss: 0.515673
Epoch 9/50, Loss: 0.487413
Epoch 10/50, Loss: 0.389086
Epoch 11/50, Loss: 0.358918
Epoch 12/50, Loss: 0.375697
Epoch 13/50, Loss: 0.298967
Epoch 14/50, Loss: 0.311738
Epoch 15/50, Loss: 0.292340
Epoch 16/50, Loss: 0.289028
Epoch 17/50, Loss: 0.298448
Epoch 18/50, Loss: 0.494038
Epoch 19/50, Loss: 0.301402
Epoch 20/50, Loss: 0.228202
Epoch 21/50, Loss: 0.263195
Epoch 22/50, Loss: 0.238447
Epoch 23/50, Loss: 0.234834
Epoch 24/50, Loss: 0.227094
Epoch 25/50, Loss: 0.244272
Epoch 26/50, Loss: 0.252135
Epoch 27/50, Loss: 0.262701
Epoch 28/50, Loss: 0.258383
Epoch 29/50, Loss: 0.226504
Epoch 30/50, Loss: 0.207507
Epoch 31/50, Loss: 0.206791
Epoch 32/50, Loss: 0.244846
Epoch 33/50, Loss: 0.229610
Epoch 34/50, Loss: 0.217835
Epoch 35/50, Loss: 0.223733
Epoch 36/5

In [18]:
mse = torch.nn.MSELoss()
mae = torch.nn.L1Loss(reduction='sum')

test_dataset = LSTMdataset(test_df, train=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
test_mse = 0.0
test_mae = 0.0
predictions = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        x_lstm = X[:, 2:].reshape(-1, 10, 2)
        x_means = X[:, :2]
        outputs = model(x_lstm, x_means)      
        predictions.append(scaler_target.inverse_transform(outputs.cpu()))
        loss_mse = mse(outputs, y)
        loss_mae = mae(outputs, y)
        test_mse += loss_mse.item() * X.size(0)
        test_mae += loss_mae.item()

avg_test_mse = test_mse / len(test_loader.dataset)
avg_test_mae = test_mae / len(test_loader.dataset)
print(f"Test MSE: {avg_test_mse:.6f}")
print(f"Test MAE: {avg_test_mae:.6f}")

Test MSE: 0.334465
Test MAE: 0.888506


In [19]:

# Suppose scaler_target is already fitted on your target columns (['cov', 'var_NVDA', 'var_SPY'])
# Get one sample from the test set
i = 5
X_sample, y_true = test_dataset[i]  # X_sample: features, y_true: normalized target

# Prepare input for model (add batch dimension)
X_sample = X_sample.unsqueeze(0).to(device)
x_lstm = X_sample[:, 2:].reshape(-1, 10, 2)
x_means = X_sample[:, :2]

# Get model prediction (normalized)
model.eval()
with torch.no_grad():
    y_pred = model(x_lstm, x_means).cpu().numpy()  # shape: (1, 3)

# Inverse transform to get un-normalized prediction
y_pred_unscaled = scaler_target.inverse_transform(y_pred)[0]  # shape: (3,)

# Inverse transform the true value for comparison
y_true_unscaled = scaler_target.inverse_transform(y_true.cpu().numpy().reshape(1, -1))[0]

print("Predicted (un-normalized):", y_pred_unscaled)
print("True (un-normalized):     ", y_true_unscaled)

Predicted (un-normalized): [7.2418683e-05 4.8994872e-04 4.5721303e-05]
True (un-normalized):      [1.1664949e-04 5.3902931e-04 3.4938363e-05]


In [20]:
test_df_r = df_r.loc["2024-01-01":]
test_df_r

,rNVDA_today,rSPY_today
Date,,
2024-01-02,-0.027341,-0.005596
2024-01-03,-0.012436,-0.008167
2024-01-04,0.009019,-0.003221
2024-01-05,0.022897,0.001370
2024-01-08,0.064281,0.014276
...,...,...
2025-05-12,0.054436,0.033047
2025-05-13,0.056341,0.006604
2025-05-14,0.041638,0.001278


In [21]:
test_df

,cov,var_SPY,mean_SPY,var_NVDA,mean_NVDA,rSPY_t-10,rNVDA_t-10,rSPY_t-9,rNVDA_t-9,rSPY_t-8,...,rSPY_t-5,rNVDA_t-5,rSPY_t-4,rNVDA_t-4,rSPY_t-3,rNVDA_t-3,rSPY_t-2,rNVDA_t-2,rSPY_t-1,rNVDA_t-1
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,0.000084,0.000042,0.001121,0.000290,0.002503,-0.001647,0.011169,0.005625,0.024279,0.006081,...,0.002010,-0.003266,0.004223,0.009195,0.001808,0.002800,0.000378,0.002125,-0.002895,0.000000
2024-01-03,0.000096,0.000048,0.000726,0.000287,-0.001348,0.005625,0.024279,0.006081,-0.009445,-0.013857,...,0.004223,0.009195,0.001808,0.002800,0.000378,0.002125,-0.002895,0.000000,-0.005596,-0.027341
2024-01-04,0.000076,0.000045,-0.000653,0.000227,-0.005020,0.006081,-0.009445,-0.013857,-0.030098,0.009482,...,0.001808,0.002800,0.000378,0.002125,-0.002895,0.000000,-0.005596,-0.027341,-0.008167,-0.012436
2024-01-05,0.000087,0.000041,-0.001584,0.000285,-0.003173,-0.013857,-0.030098,0.009482,0.018270,0.002010,...,0.000378,0.002125,-0.002895,0.000000,-0.005596,-0.027341,-0.008167,-0.012436,-0.003221,0.009019
2024-01-08,0.000128,0.000042,-0.000061,0.000542,0.002126,0.009482,0.018270,0.002010,-0.003266,0.004223,...,-0.002895,0.000000,-0.005596,-0.027341,-0.008167,-0.012436,-0.003221,0.009019,0.001370,0.022897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-12,0.000202,0.000127,0.002482,0.000471,0.005092,0.000381,-0.020539,0.006299,0.002667,0.000397,...,-0.005734,-0.005939,-0.008358,-0.002460,0.004206,0.031002,0.006968,0.002648,-0.001274,-0.006134
2025-05-13,0.000190,0.000124,0.005748,0.000545,0.012589,0.006299,0.002667,0.000397,-0.000917,0.007087,...,-0.008358,-0.002460,0.004206,0.031002,0.006968,0.002648,-0.001274,-0.006134,0.033047,0.054436
2025-05-14,0.000181,0.000126,0.005779,0.000575,0.017957,0.000397,-0.000917,0.007087,0.024697,0.014844,...,0.004206,0.031002,0.006968,0.002648,-0.001274,-0.006134,0.033047,0.054436,0.006604,0.056341


In [22]:
def get_mvp(var1,var2,cov):
    std1 = np.sqrt(var1)
    std2 = np.sqrt(var2)
    corr = cov/(std1*std2)
    #print("corr: ",corr)
    nom = var2 - (corr*std1*std2)
    den = var1 + var2 - (2*corr*std1*std2)
    return nom/den

def get_capital_values(capital,w):
    return w * capital, (1-w)*capital

def get_return_values(return1, return2, w):
    return w * return1 + (1-w)*return2

In [23]:
output = test_df[['cov', 'var_SPY', 'var_NVDA']]
output

,cov,var_SPY,var_NVDA
Date,,,
2024-01-02,0.000084,0.000042,0.000290
2024-01-03,0.000096,0.000048,0.000287
2024-01-04,0.000076,0.000045,0.000227
2024-01-05,0.000087,0.000041,0.000285
2024-01-08,0.000128,0.000042,0.000542
...,...,...,...
2025-05-12,0.000202,0.000127,0.000471
2025-05-13,0.000190,0.000124,0.000545
2025-05-14,0.000181,0.000126,0.000575


In [29]:
def run_portfolio_simulation(var_source='real', capital=1000):
    weights_nvda, weights_spy = [], []
    capital_NVDA, capital_SPY = [], []
    portfolio_values = [capital]
    nvda_asset_1 = True

    for day in range(len(test_df)):
        print(f"DAY {day}")
        row = test_df.iloc[day]

        # Update portfolio value
        if day > 0:
            w_old = weights_nvda[day - 1] if nvda_asset_1 else weights_spy[day - 1]
            r_nvda = test_df_r['rNVDA_today'][day - 1]
            r_spy = test_df_r['rSPY_today'][day - 1]
            daily_return = get_return_values(r_nvda, r_spy, w_old)
            new_val = portfolio_values[day - 1] * (1 + daily_return)
            portfolio_values.append(new_val)
            print("UPDATED PORTFOLIO VALUE:", new_val)

        # Retrieve variance & covariance
        if var_source == 'real':
            var_nvda = row['var_NVDA']
            var_spy = row['var_SPY']
            cov = row['cov']
        elif var_source == 'predicted':
            var_nvda = output[day][1]
            var_spy = output[day][2]
            cov = output[day][0]
        else:
            raise ValueError("var_source must be 'real' or 'predicted'")

        nvda_asset_1 = var_nvda >= var_spy

        # Allocation
        if nvda_asset_1:
            w = get_mvp(var_nvda, var_spy, cov)
            w_nvda, w_spy = w, 1 - w
        else:
            w = get_mvp(var_spy, var_nvda, cov)
            w_nvda, w_spy = 1 - w, w

        weights_nvda.append(w_nvda)
        weights_spy.append(w_spy)

        v_nvda, v_spy = get_capital_values(portfolio_values[day], w_nvda)
        capital_NVDA.append(v_nvda)
        capital_SPY.append(v_spy)

        # Print 
        if day == 0:
            print("NVDA ALLOCATED:", v_nvda)
            print("SPY ALLOCATED:", v_spy)
        else:
            nvda_diff = v_nvda - capital_NVDA[day - 1]
            spy_diff = v_spy - capital_SPY[day - 1]

            print(f"{'BUY' if nvda_diff > 0 else 'SELL'} NVDA: {abs(nvda_diff)}")
            print(f"ACTUAL NVDA: {v_nvda}")

            print(f"{'BUY' if spy_diff > 0 else 'SELL'} SPY: {abs(spy_diff)}")
            print(f"ACTUAL SPY: {v_spy}")

    print("FINAL PORTFOLIO VALUE:", portfolio_values[-1])
    return portfolio_values, weights_nvda, weights_spy, capital_NVDA, capital_SPY


In [30]:
oracle_result, weights_nvda_oracle, weights_spy_oracle, capital_NVDA_oracle, capital_SPY_oracle = run_portfolio_simulation(var_source='real')

predicted_result, weights_nvda_pred, weights_spy_pred, capital_NVDA_pred, capital_SPY_pred = run_portfolio_simulation(var_source='predicted')

DAY 0
NVDA ALLOCATED: -263.16643320224506
SPY ALLOCATED: 1263.166433202245
DAY 1
UPDATED PORTFOLIO VALUE: 1000.1262104098403
SELL NVDA: 71.22611854650046
ACTUAL NVDA: -334.3925517487455
BUY SPY: 71.3523289563409
ACTUAL SPY: 1334.5187621585858
DAY 2
UPDATED PORTFOLIO VALUE: 993.386151629976
BUY NVDA: 79.27925592785613
ACTUAL NVDA: -255.1132958208894
SELL SPY: 86.01931470772024
ACTUAL SPY: 1248.4994474508655
DAY 3
UPDATED PORTFOLIO VALUE: 987.0636932761489
SELL NVDA: 50.602480661228356
ACTUAL NVDA: -305.71577648211775
BUY SPY: 44.28002230740117
ACTUAL SPY: 1292.7794697582667
DAY 4
UPDATED PORTFOLIO VALUE: 981.8344789762962
BUY NVDA: 48.37947644959962
ACTUAL NVDA: -257.3363000325181
SELL SPY: 53.608690749452535
ACTUAL SPY: 1239.1707790088142
DAY 5
UPDATED PORTFOLIO VALUE: 982.9830287089216
BUY NVDA: 21.562789842508778
ACTUAL NVDA: -235.77351019000935
SELL SPY: 20.414240109883167
ACTUAL SPY: 1218.756538898931
DAY 6
UPDATED PORTFOLIO VALUE: 977.1317131177573
SELL NVDA: 26.633092194214385
AC

C:\Users\rikyf\AppData\Local\Temp\ipykernel_17868\1203576697.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  r_nvda = test_df_r['rNVDA_today'][day - 1]
C:\Users\rikyf\AppData\Local\Temp\ipykernel_17868\1203576697.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  r_spy = test_df_r['rSPY_today'][day - 1]


UPDATED PORTFOLIO VALUE: 1079.1511663191602
BUY NVDA: 17.87351031927301
ACTUAL NVDA: -393.6270305746713
SELL SPY: 8.856880309894905
ACTUAL SPY: 1472.7781968938316
DAY 332
UPDATED PORTFOLIO VALUE: 1087.3789642611448
SELL NVDA: 13.913235333850707
ACTUAL NVDA: -407.54026590852203
BUY SPY: 22.141033275835298
ACTUAL SPY: 1494.919230169667
DAY 333
UPDATED PORTFOLIO VALUE: 1088.3460086435987
SELL NVDA: 31.761322653962452
ACTUAL NVDA: -439.3015885624845
BUY SPY: 32.72836703641633
ACTUAL SPY: 1527.6475972060832
DAY 334
UPDATED PORTFOLIO VALUE: 1088.3229061085747
BUY NVDA: 90.47167963011617
ACTUAL NVDA: -348.8299089323683
SELL SPY: 90.49478216514012
ACTUAL SPY: 1437.152815040943
DAY 335
UPDATED PORTFOLIO VALUE: 1100.6237796651621
SELL NVDA: 79.945334401482
ACTUAL NVDA: -428.7752433338503
BUY SPY: 92.24620795806936
ACTUAL SPY: 1529.3990229990125
DAY 336
UPDATED PORTFOLIO VALUE: 1094.4001103537921
BUY NVDA: 247.44651062482598
ACTUAL NVDA: -181.32873270902434
SELL SPY: 253.67017993619584
ACTUAL SPY